# EXTRAÇÃO DE DADOS DE EXTRATOS DO NUBANK PARA CRIAÇÃO DE UM RELATÓRIO

In [13]:
# Importando Bibliotecas
import pandas as pd
import os

In [14]:
# Criando Variaveis para Leitura de um Diretorio
diretorio = r"C:\Users\ezequ\Scripts\02 - MEUS_PROJETOS\11 - NUBANK_EXTRATO\02_INPUTS"

In [63]:
# Criação de uma lista com todos os arquivos do diretorio
arquivos = os.listdir(diretorio)

# Cria um dataframe vazio para realizar o salvamento de varios arquivos em unico Dataframe no Loop 
df_Historica = pd.DataFrame()

# Criando Loop para acessar todos os arquivos do diretorio e unifica-los
for i in arquivos:
    path = f'{diretorio}/{i}'
    df_path = pd.read_csv(f'{path}')
    df_path['Nome do Arquivo'] = i
    df_Historica = pd.concat([df_Historica, df_path])

In [64]:
# Alteração da coluna Data para tipo de DATETIME do Pandas
df_Historica['Data'] = pd.to_datetime(df_Historica['Data'], format="%d/%m/%Y")

In [65]:
# Altera;áo da coluna Valor para o tipo FLOAT64 do Pandas
df_Historica['Valor'] = pd.to_numeric(df_Historica['Valor'])

In [99]:
# Atribua o novo valor à coluna 'Descrição' para as linhas que atendem à condição
df_Historica.loc[df_Historica['Descrição'].str.contains('Transferência enviada pelo Pix'), 'Descrição'] = 'Transferência enviada pelo Pix'
df_Historica.loc[df_Historica['Descrição'].str.contains('Transferência recebida pelo Pix'), 'Descrição'] = 'Transferência recebida pelo Pix'
df_Historica.loc[df_Historica['Descrição'].str.contains('Transferência Recebida'), 'Descrição'] = 'Transferência Recebida'
df_Historica.loc[df_Historica['Descrição'].str.contains('Reembolso recebido pelo Pix'), 'Descrição'] = 'Reembolso recebido pelo Pix'
df_Historica.loc[df_Historica['Descrição'].str.contains('Compra no débito'), 'Descrição'] = 'Compra no débito'
df_Historica.loc[df_Historica['Descrição'].str.contains('Pagamento de boleto efetuado'), 'Descrição'] = 'Pagamento de boleto efetuado'


In [158]:
# Tornando todos os elementos da Coluna em Letras Maisculas
df_Historica['Descrição'] = df_Historica['Descrição'].str.upper()

In [190]:
df_Historica.groupby(['Descrição']).agg(
    TOTAL = ('Valor', 'sum'), 
    RECEITA = ('RECEITA', 'sum'),
    RETIRADA_INVESTIMENTO = ('RETIRADA_INVESTIMENTO', 'sum'),
    CREDITO = ('CREDITO', 'sum'),
    DÉBITO = ('DÉBITO', 'sum'),
    GASTOS = ('GASTOS', 'sum'),
    INVESTIMENTO = ('INVESTIMENTO', 'sum'),
    ).reset_index()

,Descrição,TOTAL,RECEITA,RETIRADA_INVESTIMENTO,CREDITO,DÉBITO,GASTOS,INVESTIMENTO
0,APLICAÇÃO FUNDO - NU RESERVA IMEDIATA,-8402.41,0.00,0.00,0.00,0.00,0.00,-8402.41
1,APLICAÇÃO RDB,-50330.13,0.00,0.00,0.00,0.00,0.00,-50330.13
2,COMPRA NO DÉBITO,-5567.26,0.00,0.00,0.00,-5567.26,0.00,0.00
3,PAGAMENTO DE BOLETO EFETUADO,-3068.40,0.00,0.00,-3068.40,0.00,-3068.40,0.00
4,PAGAMENTO DE FATURA,-14545.06,0.00,0.00,-14545.06,0.00,-14545.06,0.00
5,RECARGA DE CELULAR,-235.00,0.00,0.00,0.00,0.00,-235.00,0.00
6,REEMBOLSO RECEBIDO PELO PIX,595.11,595.11,0.00,0.00,0.00,0.00,0.00
7,RESGATE FUNDO - NU RESERVA IMEDIATA,3305.14,0.00,3305.14,0.00,0.00,0.00,0.00
8,RESGATE RDB,33690.32,0.00,33690.32,0.00,0.00,0.00,0.00
9,TRANSFERÊNCIA ENVIADA PELO PIX,-28603.59,0.00,0.00,0.00,0.00,-28603.59,0.00


In [184]:
# Criando uma nova coluna 'RECEITA' com todos os valores que entraram na conta
# Criando uma tabela condicional para identificação dos elementos com os valores entrantes na conta
condition = df_Historica['Descrição'].str.contains('TRANSFERÊNCIA RECEBIDA', case=False) | df_Historica['Descrição'].str.contains('REEMBOLSO RECEBIDO PELO PIX', case=False)
# Aplicando a condição em uma nova Coluna
df_Historica['RECEITA'] = df_Historica.loc[condition, 'Valor']
df_Historica['RECEITA'] = df_Historica['RECEITA'].fillna(0)

In [185]:
# Criando uma nova coluna 'Resgate' com todos os valores que sairam dos Investimentos realizados na conta
# Criando uma tabela condicional para identificação dos valores que sairam dos Investimentos realizados na conta
condition = df_Historica['Descrição'].str.contains('RESGATE', case=False)
# Aplicando a condição em uma nova Coluna
df_Historica['RETIRADA_INVESTIMENTO'] = df_Historica.loc[condition, 'Valor']
df_Historica['RETIRADA_INVESTIMENTO'] = df_Historica['RETIRADA_INVESTIMENTO'].fillna(0)

In [186]:
# Criando uma nova coluna 'CREDITO' com todos os de Pagamentos realizado naquele mês
# Criando uma tabela condicional para identificação dos Pagamentos realizado naquele mês
condition = df_Historica['Descrição'].str.contains('PAGAMENTO', case=False)
# Aplicando a condição em uma nova Coluna
df_Historica['CREDITO'] = df_Historica.loc[condition, 'Valor']  
df_Historica['CREDITO'] = df_Historica['CREDITO'].fillna(0)

In [187]:
# Criando uma nova coluna 'DÉBITO' com todos as compras realizadas 
# Criando uma tabela condicional para identificação das compras realizadas
condition = df_Historica['Descrição'].str.contains('DÉBITO', case=False)
# Aplicando a condição em uma nova Coluna
df_Historica['DÉBITO'] = df_Historica.loc[condition, 'Valor']
df_Historica['DÉBITO'] = df_Historica['DÉBITO'].fillna(0)

In [188]:
# Criando uma nova coluna 'GASTOS' com todos os valores de gastos realizados sem exceção
# Criando uma tabela condicional para identificação de valores de gastos realizados sem exceção
condition = df_Historica['Descrição'].str.contains('PAGAMENTO', case=False) | df_Historica['Descrição'].str.contains('ENVIADA', case=False) | df_Historica['Descrição'].str.contains('RECARGA', case=False)
# Aplicando a condição em uma nova Coluna
df_Historica['GASTOS'] = df_Historica.loc[condition, 'Valor']
df_Historica['GASTOS'] = df_Historica['GASTOS'].fillna(0) 

In [189]:
# Criando uma nova coluna 'INVESTIMENTO' com todos os valores de INVESTIMENTOS
# Criando uma tabela condicional para identificação de valores de INVESTIMENTOS
condition = df_Historica['Descrição'].str.contains('APLICAÇÃO', case=False)
# Aplicando a condição em uma nova Coluna
df_Historica['INVESTIMENTO'] = df_Historica.loc[condition, 'Valor']  
df_Historica['INVESTIMENTO'] = df_Historica['INVESTIMENTO'].fillna(0)

In [192]:
df_Historica['TOTAL'] = df_Historica['Valor']

In [194]:
df_Historica = df_Historica[['Data','Descrição','RECEITA','RETIRADA_INVESTIMENTO','CREDITO','DÉBITO','GASTOS','INVESTIMENTO','TOTAL']]

In [196]:
df_Historica.to_excel(r'C:\Users\ezequ\Scripts\02 - MEUS_PROJETOS\11 - NUBANK_EXTRATO\03_OUTPUT\FINANCAS.xlsx', index=False)